In [1]:
import argparse
import os
import pickle
import sys

import numpy as np
import scanpy as sc
import torch


from scvi._settings import settings
from scvi.model import SCVI, TOTALVI
from scvi.data import synthetic_iid

from contrastive_vi.data.dataloaders.contrastive_dataloader import ContrastiveDataLoader
from contrastive_vi.module.total_contrastive_vi import TotalContrastiveVIModule

from scripts import constants

Global seed set to 0


In [2]:
n_genes = 50
n_proteins = 4
adata = synthetic_iid(
    run_setup_anndata=False,
    n_batches=2,  # Same number of cells in each batch.
    n_genes=n_genes,
    n_proteins=n_proteins,
)
# Make number of cells unequal across batches to test edge cases.
adata = adata[:-3, :]
adata.layers["raw_counts"] = adata.X.copy()

/homes/gws/clin25/miniconda3/envs/contrastive-vi-env/lib/python3.9/site-packages/scvi/data/_built_in_data/_synthetic.py:31: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = AnnData(data)


In [3]:
TOTALVI.setup_anndata(
    adata,
    batch_key="batch",
    layer="raw_counts",
    protein_expression_obsm_key="protein_expression"
)

INFO     Using batches from adata.obs["batch"]                                               
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["raw_counts"]                                          
INFO     Using protein expression from adata.obsm['protein_expression']                      
INFO     Generating sequential protein names                                                 
INFO     Successfully registered anndata object containing 397 cells, 50 vars, 2 batches, 1  
         labels, and 4 proteins. Also registered 0 extra categorical covariates and 0 extra  
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


In [4]:
background_indices = (
    adata.obs.index[(adata.obs["batch"] == "batch_0")]
    .astype(int)
    .tolist()
)
target_indices = (
    adata.obs.index[(adata.obs["batch"] == "batch_1")]
    .astype(int)
    .tolist()
)

In [5]:
dataloader = ContrastiveDataLoader(
    adata,
    background_indices,
    target_indices,
    batch_size=32,
    shuffle=False,
)
for batch in dataloader:
    pass

In [6]:
total_contrastive_vi_module = TotalContrastiveVIModule(
    n_input_genes=n_genes,
    n_input_proteins=n_proteins,
    n_hidden=12,
    n_background_latent=8,
    n_salient_latent=4,
)

In [7]:
inference_input = total_contrastive_vi_module._get_inference_input(batch)
background_inference_input = inference_input["background"]
target_inference_input = inference_input["target"]

In [8]:
generic_inference_output = total_contrastive_vi_module._generic_inference(
    **background_inference_input,
)

In [9]:
inference_outputs = total_contrastive_vi_module.inference(**inference_input)

In [10]:
generative_input = total_contrastive_vi_module._get_generative_input(
    batch, inference_outputs
)

In [11]:
generic_generative_input = generative_input["background"]

In [12]:
generic_generative_input.keys()

dict_keys(['batch_index', 'z', 's', 'library_gene'])

In [13]:
generic_generative_outputs = total_contrastive_vi_module._generic_generative(
    **generic_generative_input
)

In [14]:
generative_outputs = total_contrastive_vi_module.generative(**generative_input)

In [15]:
generative_outputs["background"].keys()

dict_keys(['px_', 'py_', 'log_pro_back_mean'])

In [16]:
background_tensors = batch["background"]

In [17]:
loss_recorder = total_contrastive_vi_module.loss(
    batch, inference_outputs, generative_outputs
)

In [18]:
loss_recorder.loss

tensor(467.8974, grad_fn=<AddBackward0>)

In [19]:
loss_recorder.reconstruction_loss.shape

torch.Size([5])